In [1]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [29]:
s=Service(executable_path="C:\\chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get("https://www.tomsguide.com/topics/smartphones/reviews")
time.sleep(30)

In [30]:
def extract_name(string):
    new_string = ""
    for word in string.split():
        if "review" not in word.lower():
            new_string += word + " "
        else:
            break
    return new_string.strip()

In [31]:
review_dict = {"Phone Name" : [], "Source" : "Toms Guide", "URL" : [], "Review" : []}
i = 1
j = 2
while True:
    try:
        driver.execute_script("window.scroll(0, 700)")
        review_link = driver.find_element(By.XPATH, f'//*[@id="content"]/section[2]/div/div[{i}]')
        a_tag = driver.find_element(By.XPATH, f'//*[@id="content"]/section[2]/div/div[{i}]/a')
        phone_name = review_link.text
        if "review" not in phone_name.lower():
            phone_name = "DELETE_THIS"
        phone_link = a_tag.get_attribute('href')
        review_link.click()
        time.sleep(2)
        review_data = driver.find_element(By.ID, 'article-body')
        review_paras = review_data.find_elements(By.TAG_NAME, 'p')
        final_review = ""
        for para in review_paras:
            final_review += para.text
        final_review = final_review.strip().replace('\n', " ")
        review_dict["Phone Name"].append(extract_name(phone_name))
        review_dict["URL"].append(phone_link)
        review_dict["Review"].append(final_review)
        driver.back()
        i += 1
        review_df = pd.DataFrame(review_dict)
    except:
        if j < 9:
            driver.execute_script("window.scroll(0, 1080)")
            next_page = driver.find_element(By.XPATH, f'//*[@id="content"]/section[2]/div/div[31]/ul/li[{j}]/a')
            next_page.click()
            time.sleep(1)
            j += 1
            i = 1
        else:
            break

In [32]:
review_df

,Phone Name,Source,URL,Review
0,Android 12,Toms Guide,https://www.tomsguide.com/reviews/android-12,• New lock screen • A new theming system • Qu...
1,Google Pixel 6,Toms Guide,https://www.tomsguide.com/reviews/google-pixel-6,Screen size: 6.4-inch OLED (2400 x 1080) Refre...
2,Google Pixel 6 Pro,Toms Guide,https://www.tomsguide.com/reviews/google-pixel...,Price: Starting at $899 Android version: 12 Di...
3,Honor 50,Toms Guide,https://www.tomsguide.com/reviews/honor-50,"Starting price: £449 Display: 6.57-inch OLED, ..."
4,Samsung Galaxy S21 Ultra,Toms Guide,https://www.tomsguide.com/reviews/samsung-gala...,The Samsung Galaxy S21 Ultra is the best Andro...
...,...,...,...,...
219,Moto Z2 Play,Toms Guide,"https://www.tomsguide.com/us/moto-z2-play,revi...",LG may have abandoned its family of mods for t...
220,Samsung Galaxy S8 Active,Toms Guide,https://www.tomsguide.com/us/samsung-galaxy-s8...,Samsung's Galaxy S8 is the best smartphone you...
221,ZTE Blade Z Max,Toms Guide,"https://www.tomsguide.com/us/zte-blade-z-max,r...",Samsung is preparing to ship the Galaxy Note 8...
222,Asus ZenFone AR,Toms Guide,"https://www.tomsguide.com/us/asus-zenfone-ar,r...",When it comes to augmented and virtual reality...


In [54]:
final_df = review_df.drop(review_df.index[review_df['Phone Name'] == 'DELETE_THIS'])
final_df

,Phone Name,Source,URL,Review
0,Android 12,Toms Guide,https://www.tomsguide.com/reviews/android-12,• New lock screen • A new theming system • Qu...
1,Google Pixel 6,Toms Guide,https://www.tomsguide.com/reviews/google-pixel-6,Screen size: 6.4-inch OLED (2400 x 1080) Refre...
2,Google Pixel 6 Pro,Toms Guide,https://www.tomsguide.com/reviews/google-pixel...,Price: Starting at $899 Android version: 12 Di...
3,Honor 50,Toms Guide,https://www.tomsguide.com/reviews/honor-50,"Starting price: £449 Display: 6.57-inch OLED, ..."
4,Samsung Galaxy S21 Ultra,Toms Guide,https://www.tomsguide.com/reviews/samsung-gala...,The Samsung Galaxy S21 Ultra is the best Andro...
...,...,...,...,...
219,Moto Z2 Play,Toms Guide,"https://www.tomsguide.com/us/moto-z2-play,revi...",LG may have abandoned its family of mods for t...
220,Samsung Galaxy S8 Active,Toms Guide,https://www.tomsguide.com/us/samsung-galaxy-s8...,Samsung's Galaxy S8 is the best smartphone you...
221,ZTE Blade Z Max,Toms Guide,"https://www.tomsguide.com/us/zte-blade-z-max,r...",Samsung is preparing to ship the Galaxy Note 8...
222,Asus ZenFone AR,Toms Guide,"https://www.tomsguide.com/us/asus-zenfone-ar,r...",When it comes to augmented and virtual reality...


In [55]:
final_df = final_df.reset_index()

In [57]:
final_df.to_csv("toms_guide.csv", index=False)

In [12]:
final_df = pd.read_csv("toms_guide.csv")

In [13]:
final_df.drop("Unnamed: 0", inplace=True, axis=1)

In [14]:
final_df

,Phone Name,Source,URL,Review
0,Android 12,Toms Guide,https://www.tomsguide.com/reviews/android-12,• New lock screen • A new theming system • Qu...
1,Google Pixel 6,Toms Guide,https://www.tomsguide.com/reviews/google-pixel-6,Screen size: 6.4-inch OLED (2400 x 1080) Refre...
2,Google Pixel 6 Pro,Toms Guide,https://www.tomsguide.com/reviews/google-pixel...,Price: Starting at $899 Android version: 12 Di...
3,Honor 50,Toms Guide,https://www.tomsguide.com/reviews/honor-50,"Starting price: £449 Display: 6.57-inch OLED, ..."
4,Samsung Galaxy S21 Ultra,Toms Guide,https://www.tomsguide.com/reviews/samsung-gala...,The Samsung Galaxy S21 Ultra is the best Andro...
...,...,...,...,...
211,Moto Z2 Play,Toms Guide,"https://www.tomsguide.com/us/moto-z2-play,revi...",LG may have abandoned its family of mods for t...
212,Samsung Galaxy S8 Active,Toms Guide,https://www.tomsguide.com/us/samsung-galaxy-s8...,Samsung's Galaxy S8 is the best smartphone you...
213,ZTE Blade Z Max,Toms Guide,"https://www.tomsguide.com/us/zte-blade-z-max,r...",Samsung is preparing to ship the Galaxy Note 8...
214,Asus ZenFone AR,Toms Guide,"https://www.tomsguide.com/us/asus-zenfone-ar,r...",When it comes to augmented and virtual reality...


In [15]:
final_df.to_csv("CSVs/toms_guide.csv")